In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import mysql.connector
# KSRTC BUS ROUTE EXTRACTION
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
driver.get('https://www.redbus.in/online-booking/ksrtc-kerala'); #url to scrape
time.sleep(10) # Let the user actually see something!
driver.maximize_window()
def extract_bus_routes():
    routes=[]
    body=driver.find_element(By.TAG_NAME,'body')
    for i in range(5):
        for j in range(2):
            body.send_keys(Keys.PAGE_DOWN)
    time.sleep(10)
    route_details=driver.find_elements(By.CLASS_NAME,'route_details')
    for route in route_details:
        route_info=route.find_element(By.CLASS_NAME,'route')
        route_text=route_info.text
        source,destination=route_text.split(' to ')
        routes.append({"source":source,"destination":destination})
    return routes
all_routes=[]

pagination=driver.find_element(By.CLASS_NAME,'DC_117_paginationTable')
page_buttons=pagination.find_elements(By.TAG_NAME,'div')
for page_button in page_buttons:
    try:
        page_button.click()
        route_on_page=extract_bus_routes()
        all_routes.extend(route_on_page)
    except Exception as e:
        print(f"An error occured:{e}")
        break
driver.quit()
for route in all_routes:
    print(route)
# connecting my sql data base
db = mysql.connector.connect(
        host='127.0.0.1',
        port='3306',
        user='root',
        password='123456789',
        database='redbus_db'
    )   
if db:
        print('Connected to MySQL database')
else:
        print('Error connecting to MySQL database')
cursor= db.cursor()
cursor.execute("""CREATE TABLE IF NOT EXISTS bus_routes(
               ID integer auto_increment primary key,
               STATE TEXT,
               SOURCE text,
               DESTINATION text,
               BUS_NAME text,
               BUS_TYPE text,  
               DEPARTING_TIME text,
               ARRIVING_TIME text,
               DURATION text,
               STAR_RATING float,
               PRICE decimal,
               SEATS_AVAILABLE text,
               BUS_CATEGORY text,
               AC_TYPE text,
               STAR_RANGE text)"""
               )

# Scraping of Bus Details
driver = webdriver.Chrome()
def scrape_bus_details(source,destination):
    driver.get('https://www.redbus.in/online-booking/ksrtc-kerala'); #url to scrape
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body')))
    driver.maximize_window()
    time.sleep(5) # Let the user actually see something!
    source_input= driver.find_element (By.ID,'txtSource')
    time.sleep(2)
    source_input.click()
    driver.execute_script("arguments[0].value= '';",source_input)
    source_input.send_keys(source)
    time.sleep(5)
    source_input.send_keys(Keys.ENTER)
    time.sleep(2)
    dest_input= driver.find_element (By.ID,'txtDestination')
    time.sleep(2)
    dest_input.click()
    driver.execute_script("arguments[0].value= '';",dest_input)
    dest_input.send_keys(destination)
    time.sleep(5)
    dest_input.send_keys(Keys.ENTER)
    time.sleep(2)
    select_date="23-Jul-2024"
    dates=select_date.split("-")
    Calendar=driver.find_element(By.ID, 'txtOnwardCalendar').click()
    date=driver.find_elements(By.XPATH,'//*[@id="rb-calendar"]//li')

    for ele in date:
        if ele.text==dates[0]:
            print(ele.text)
            ele.click()
            break 

    Search_button=driver.find_element (By.CSS_SELECTOR, "button[class='D120_search_btn_v2 searchBuses']").click()
    time.sleep(20)
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body')))
    filters=driver.find_elements(By.XPATH,'//div[@class="filter-details f-12 d-color"]')
    for filter in filters:
        reset=driver.find_element(By.XPATH,'//*[@id="filter-block"]/div/div[1]/span')
        reset.click()
    body=driver.find_element(By.TAG_NAME,'body')
    for i in range(50):
        for j in range(2):
            body.send_keys(Keys.PAGE_DOWN)

    govt_bus_detail=[]
    bus_items=driver.find_elements(By.XPATH,'//div[@class="clearfix bus-item"]')
    WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,'//div[@class="clearfix bus-item"]')))
    time.sleep(10)
    for bus in bus_items:
        try:
            bus_name = bus.find_element(By.XPATH, './/div[@class="travels lh-24 f-bold d-color"]').text
            if "KSRTC" in bus_name:
                bus_category="Government"
            else:
                bus_category="private" 
            bus_type = bus.find_element(By.XPATH, './/div[@class="bus-type f-12 m-top-16 l-color evBus"]').text
            if "Non" in bus_type:
                AC_type="NON AC"
            else:
                AC_type="AC"
        
            departure_time = bus.find_element(By.XPATH, './/div[@class="dp-time f-19 d-color f-bold"]').text
            arrival_time = bus.find_element(By.XPATH, './/div[@class="bp-time f-19 d-color disp-Inline"]').text
            duration=bus.find_element(By.XPATH,'//div[@class="dur l-color lh-24"]').text
            star_rating=bus.find_element(By.XPATH,'.//div[@class="rating-sec lh-24"]').text
            star=float(star_rating)
            if star<2:
                star_range='1-2'
            elif 2<=star<3:
                star_range='2-3'
            elif 3<=star<4:
                star_range='3-4'
            else:
                star_range='4 above'
            fare = bus.find_element(By.XPATH, ".//span[contains(@class,'f-19')]").text
            seats_avail= bus.find_element(By.XPATH,".//div[contains(@class,'seat-left')]").text
            seats=seats_avail.split(' ')[0]
            state="Kerala"
            govt_bus_detail.append((bus_name, bus_type, departure_time, arrival_time,duration,star_rating,fare,seats,bus_category,AC_type,star_range))
            cursor.execute("""INSERT INTO bus_routes(STATE,SOURCE,DESTINATION,BUS_NAME,BUS_TYPE,DEPARTING_TIME,ARRIVING_TIME,DURATION,STAR_RATING,PRICE,SEATS_AVAILABLE,BUS_CATEGORY,AC_TYPE,STAR_RANGE
        )VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""",(state,source,destination,bus_name,bus_type,departure_time,arrival_time,duration,star_rating,fare,seats,bus_category,AC_type,star_range))
            db.commit()
        except Exception as e:
                print(f"An error occured:{e}")
                continue
    print(govt_bus_detail)
for route in all_routes:
    if "source" in route and "destination" in route:
            scrape_bus_details(route["source"],route["destination"])
     
driver.quit()
cursor.close()
db.close()

